# Lab 6: Solución de sistemas de ecuaciones lineales

In [1]:
import numpy as np
import scipy as sc
import math
import matplotlib.pyplot as plt

## 1)

Escribir `soltrsup` y `soltrinf` que resuelvan el sistema lineal $ Ax = b $, donde $ A $ es una matriz triangular (superior e inferior, respectivamente). La entrada debe ser $ (A, b) $ con $ A \in \mathbb{R}^{nxn} $ matriz triangular y $ b \in \mathbb{R} $, y la salida debe ser la solución $ x $. Se debe imprimir un mensaje de error si la matriz es singular.

In [65]:
def soltrinf(A, b):
    
    # comenzamos verificando que no sea singular
    # es decir, que su determinante es distinto de cero
    assert np.linalg.det(A) != 0, 'A no puede ser singular'
    assert A.shape[0] == b.shape[0], 'A y b deben tener la misma altura'
    
    # definimos N en base a la dimensión de la matriz A
    N = A.shape[0]
    
    # caculamos el x
    x = np.zeros_like(b, dtype=np.float32)
    for i in range(N):
        # buscamos la suma de los productos
        # entre la sección triangular inferior
        # y lo que calculamos ya de X
        s = sum([A[i, j] * x[j] for j in range(i)])
        
        # buscamos el resultado final de x[i]
        x[i] = (b[i] - s) / A[i, i]
        
    return x


def soltsup(A, b):
    
    # comenzamos verificando que no sea singular
    # es decir, que su determinante es distinto de cero
    assert np.linalg.det(A) != 0, 'A no puede ser singular'
    assert A.shape[0] == b.shape[0], 'A y b deben tener la misma altura'
    
    # definimos N en base a la dimensión de la matriz A
    N = A.shape[0]
    
    # caculamos el x
    x = np.zeros_like(b, dtype=np.float32)
    for i in reversed(range(N)):
        
        # buscamos la suma de los productos
        # entre la sección triangular superior
        # y lo que calculamos de X
        s = sum([A[i, j] * x[j] for j in range(i + 1, N)])
        
        # buscamos el resultado final de x[i]
        x[i] = (b[i] - s) / A[i, i]
        
    return x


Probamos las funciones con matrices conocidas ([https://matrixcalc.org/en/slu.html]()):

$$
\left(\begin{array}{cc}
1 & 0 & 0 \\
5 & 5 & 0 \\
3 & 2 & 1
\end{array}\right)
\left(\begin{array}{cc}
2 \\
-7/5 \\
-6/5
\end{array}\right)
=
\left(\begin{array}{cc}
2 \\
3 \\
2
\end{array}\right)
$$ 

In [66]:
A = np.stack([[1, 0, 0],
              [5, 5, 0],
              [3, 2, 1]])
b = np.array([2, 3, 2])

x = soltrinf(A, b)
x

array([ 2. , -1.4, -1.2], dtype=float32)

$$
\left(\begin{array}{cc}
1 & 5 & 3 \\
0 & 5 & 5 \\
0 & 0 & 1
\end{array}\right)
\left(\begin{array}{cc}
3 \\
-7/5 \\
2
\end{array}\right)
=
\left(\begin{array}{cc}
2 \\
3 \\
2
\end{array}\right)
$$ 

In [67]:
A = np.stack([[1, 5, 3],
              [0, 5, 5],
              [0, 0, 1]])
b = np.array([2, 3, 2])

x = soltsup(A, b)
x

array([ 3. , -1.4,  2. ], dtype=float32)

## 2) 

### a) 

Escribir una función llamada `egauss` que implemente el método de la eliminación Gaussiana. Este método nos ayuda a simplificar sistemas representados con matrices triangulares superiores (no necesariamente no singulares), de manera que puedan ser resueltos por el método descripto en el ejercicio anterior.

Por esta razón, la entrada de la función debería ser una matrix y un vector, y se devuelven $ U, y $, correspondientes a $ A, b $ transformados.

In [76]:
def egauss(A, b):
    
    # definimos N en base a la dimensión de A
    N = A.shape[0]
    
    # copiamos los datos de entrada
    # los cuales iremos modificando para simplificar
    # el sistema
    U, y = A.copy(), b.copy()
    U, y = U.astype(np.float32), y.astype(np.float32)
    
    for k in range(N - 1):
        for i in range(k + 1, N):            
            assert A[k, k] != 0, f'Diag Cero:  A{k}{k} = 0'
            
            # calculamos los nuevos valores
            # de nuestra matriz / vector
            m = U[i, k] / U[k, k]

            for j in range(k + 1, N):
                U[i, j] = U[i, j] - m * U[k, j]

            y[i] = y[i] - m*y[k]
    
    # convertimos a cero todos los valores
    # por debajo de la diagonal de U 
    # ya que queremos una matriz triangular superior no singular
    U = np.triu(U)

    return U, y


## b)

Escribir una función llamada `soleg` que resuelva sistemas lineales $ Ax = b $ utilizando eliminación Gaussiana y resolviendo el sistema triangular superior.

In [78]:

def soleg(A, b):
    
    # conseguimos la simplificación
    U, y = egauss(A, b)
    
    # resolvemos el sistema triangular superior
    x = soltsup(U, y)
    
    return x

Probamos una solución conocida:

$$
\left(\begin{array}{cc}
6 & -2 & 2 & 4 \\
12 & -8 & 6 & 10 \\
3 & -13 & 9 & 3 \\
-6 & 4 & 1 & -18
\end{array}\right)
\left(\begin{array}{cc}
1 \\
-3 \\
-2 \\
1
\end{array}\right)
=
\left(\begin{array}{cc}
12 \\
34 \\
27 \\
-38
\end{array}\right)
$$ 

In [79]:
# construir las matrices
A = np.stack([[6, -2, 2, 4],
              [12, -8, 6, 10],
              [3, -13, 9, 3],
              [-6, 4, 1, -18]])

b = np.array([12, 34, 27, -38])

# usamos soleg

x = soleg(A, b)

x

array([ 1., -3., -2.,  1.], dtype=float32)

## 3)

Escribir una función llamada `sollu` que resuelva sistemas lineales $ Ax = b $ usando descomposición LU con pivoteo (para obtener dicha descomposición investigar el subpaquete de la librería scipy: `linalg`) para luego resolver $ Ly = P^-1 b $ (¿Cómo se puede obtener la inversa de una matriz de pivoteo?) y $ Ux = y $ usando `soltrinf` y `soltrsup`. la salida debe ser la solución $x$ y debe tener de entrada $ (A, b) $ con $ A \in \mathbb R^{nxn} $ y $ b \in \mathbb R^n $. 

In [ ]:
def sollu(A, b):
    
    